### Read me
This notebook will will be mainly used for *the capstone project*

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import time as tm
from datetime import datetime as dt
import requests

##### Load the wikipage and clean the dataframe

In [2]:
SKT = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df1 = SKT[0]
df1 = df1.dropna().reset_index(drop=True)
df1.columns = ['Postal Code','Borough','Neighborhood']
df1['Neighborhood'] = df1['Neighborhood'].str.replace(' / ', ', ')
df1.sample(10)

,Postal Code,Borough,Neighborhood
43,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
101,M7Y,East Toronto,Business reply mail Processing CentrE
19,M1E,Scarborough,"Guildwood, Morningside, West Hill"
28,M2H,North York,Hillcrest Village
100,M4Y,Downtown Toronto,Church and Wellesley
88,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
75,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
103,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
2,M4A,North York,Victoria Village
52,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [3]:
df1.shape

(104, 3)

##### Load the geo-data

In [4]:
df1_geo = pd.read_csv('Geospatial_Coordinates.csv', index_col=0)
df1_geo.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


##### Connect geo-data to the previus datafram

In [5]:
df_nbhood = df1.join(df1_geo, on='Postal Code')
df_nbhood.sample(7)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
26,M6G,Downtown Toronto,Christie,43.669542,-79.422564
49,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
97,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
102,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
79,M1S,Scarborough,Agincourt,43.794200,-79.262029
58,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
86,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577


##### Selecting boroughs that contain 'Toronto'

In [6]:
df_borough_Tor = df_nbhood.loc[df_nbhood['Borough'].str.contains('Toronto', regex=False)].reset_index(drop=True)
df_borough_Tor

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
